In [1]:
import torch
import gym
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import Adam

In [2]:
env = gym.make('CartPole-v0')

In [3]:
class PolicyNet():
    def __init__(self,num_state,num_action,num_hidden=128,lr=0.001):
        self.model = nn.Sequential(
                     nn.Linear(num_state,num_hidden),
                     nn.ReLU(),
                     nn.Linear(num_hidden,num_action),
                     nn.Softmax(),)
        self.optimizer = Adam(self.model.parameters(),lr)
    def predict(self,s):
        return self.model(torch.Tensor(s))
    def update(self,Return_Values,log_probs):
        policy_grad = []
        for log_prob,Gt in zip(log_probs,Return_Values):
            policy_grad.append(-log_prob * Gt)
        loss = torch.stack(policy_grad).sum()
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    def get_action(self,s):
        probs = self.predict(s)
        action = torch.multinomial(probs,1).item()
        log_prob = torch.log(probs[action])
        return action,log_prob
        
        

In [4]:
class ValueNet():
    def __init__(self,num_state,num_hidden =128,lr = 0.05):
        self.model = nn.Sequential(
                       nn.Linear(num_state,num_hidden),
                       nn.ReLU(),
                       nn.Linear(num_hidden,1)
        )
        self.loss = nn.MSELoss()
        self.optimizer = Adam(self.model.parameters(),lr)
    def update(self,s,y):
        y_pred = self.model(torch.Tensor(s))
        loss = self.loss(y_pred,Variable(torch.Tensor(y)))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    def predict(self,s):
        with torch.no_grad():
            return self.model(torch.Tensor(s))
        

In [5]:
def Reinforce_with_baseline(env,policy_net,value_net,num_epochs,gamma=0.99):
    for epoch in range(num_epochs):
        log_probs = []
        states = []
        rewards = []
        state = env.reset()
        
        while True:
            states.append(state)
            action,log_prob = policy_net.get_action(state)
            next_state,reward,done,_ = env.step(action)
            total_reward_episode[epoch] += reward
            log_probs.append(log_prob)
            rewards.append(reward)
            
            if done:
                Gt = 0
                pw = 0
                returns = []
                for k in range(len(states)):
                    Gt += gamma ** pw * rewards[k]
                    pw += 1
                    returns.append(Gt)
                #returns = returns[::-1]
                returns = torch.tensor(returns)
                baseline_values = value_net.predict(states)
                Return_Values = returns - baseline_values
                value_net.update(states,returns)
                policy_net.update(Return_Values,log_probs)
                print(f"Episode: {epoch}, total_reward:{total_reward_episode[epoch]}")
                break
            state = next_state
                

In [6]:
num_state = env.observation_space.shape[0]
num_action = env.action_space.n
num_hidden_p = 64
lr_p = 0.003
policy_net = PolicyNet(num_state, num_action, num_hidden_p, lr_p)

num_hidden_v = 64
lr_v = 0.003
value_net = ValueNet(num_state, num_hidden_v, lr_v)

num_episode = 2000
gamma = 0.9
total_reward_episode = [0] * num_episode

Reinforce_with_baseline(env, policy_net, value_net, num_episode, gamma)


/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
<ipython-input-4-3819a5188e2e>:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  return self.model(torch.Tensor(s))
/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([15])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Episode: 0, total_reward:15.0
Episode: 1, total_reward:21.0
Episode: 2, total_reward:11.0
Episode: 3, total_reward:17.0
Episode: 4, total_reward:59.0
Episode: 5, total_reward:27.0


/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([21])) that is different to the input size (torch.Size([21, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([11])) that is different to the input size (torch.Size([11, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([17])) that is different to the input size (torch.Size([17, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

Episode: 6, total_reward:21.0
Episode: 7, total_reward:15.0
Episode: 8, total_reward:12.0
Episode: 9, total_reward:10.0
Episode: 10, total_reward:13.0
Episode: 11, total_reward:32.0
Episode: 12, total_reward:12.0
Episode: 13, total_reward:18.0


/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([12, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([13])) that is different to the input size (torch.Size([13, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

...

Episode: 1457, total_reward:200.0
Episode: 1458, total_reward:200.0
Episode: 1459, total_reward:200.0
Episode: 1460, total_reward:200.0
Episode: 1461, total_reward:200.0
Episode: 1462, total_reward:200.0
Episode: 1463, total_reward:200.0
Episode: 1464, total_reward:200.0
Episode: 1465, total_reward:200.0
Episode: 1466, total_reward:200.0
Episode: 1467, total_reward:200.0
Episode: 1468, total_reward:200.0
Episode: 1469, total_reward:200.0
Episode: 1470, total_reward:200.0
Episode: 1471, total_reward:193.0


/home/user/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([193])) that is different to the input size (torch.Size([193, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Episode: 1472, total_reward:200.0
Episode: 1473, total_reward:200.0
Episode: 1474, total_reward:200.0
Episode: 1475, total_reward:200.0
Episode: 1476, total_reward:200.0
Episode: 1477, total_reward:200.0
Episode: 1478, total_reward:200.0
Episode: 1479, total_reward:200.0
Episode: 1480, total_reward:200.0
Episode: 1481, total_reward:200.0
Episode: 1482, total_reward:200.0
Episode: 1483, total_reward:200.0
Episode: 1484, total_reward:200.0
Episode: 1485, total_reward:200.0
Episode: 1486, total_reward:200.0
Episode: 1487, total_reward:75.0
Episode: 1488, total_reward:200.0
Episode: 1489, total_reward:200.0
Episode: 1490, total_reward:200.0
Episode: 1491, total_reward:200.0
Episode: 1492, total_reward:200.0
Episode: 1493, total_reward:200.0
Episode: 1494, total_reward:200.0
Episode: 1495, total_reward:200.0
Episode: 1496, total_reward:200.0
Episode: 1497, total_reward:200.0
Episode: 1498, total_reward:200.0
Episode: 1499, total_reward:200.0
Episode: 1500, total_reward:200.0
Episode: 1501, 

Episode: 1714, total_reward:200.0
Episode: 1715, total_reward:200.0
Episode: 1716, total_reward:200.0
Episode: 1717, total_reward:200.0
Episode: 1718, total_reward:200.0
Episode: 1719, total_reward:200.0
Episode: 1720, total_reward:200.0
Episode: 1721, total_reward:200.0
Episode: 1722, total_reward:200.0
Episode: 1723, total_reward:200.0
Episode: 1724, total_reward:200.0
Episode: 1725, total_reward:165.0
Episode: 1726, total_reward:200.0
Episode: 1727, total_reward:200.0
Episode: 1728, total_reward:200.0
Episode: 1729, total_reward:200.0
Episode: 1730, total_reward:200.0
Episode: 1731, total_reward:200.0
Episode: 1732, total_reward:200.0
Episode: 1733, total_reward:200.0
Episode: 1734, total_reward:200.0
Episode: 1735, total_reward:200.0
Episode: 1736, total_reward:200.0
Episode: 1737, total_reward:200.0
Episode: 1738, total_reward:200.0
Episode: 1739, total_reward:200.0
Episode: 1740, total_reward:200.0
Episode: 1741, total_reward:200.0
Episode: 1742, total_reward:200.0
Episode: 1743,

Episode: 1955, total_reward:200.0
Episode: 1956, total_reward:200.0
Episode: 1957, total_reward:200.0
Episode: 1958, total_reward:200.0
Episode: 1959, total_reward:200.0
Episode: 1960, total_reward:200.0
Episode: 1961, total_reward:200.0
Episode: 1962, total_reward:200.0
Episode: 1963, total_reward:200.0
Episode: 1964, total_reward:200.0
Episode: 1965, total_reward:200.0
Episode: 1966, total_reward:200.0
Episode: 1967, total_reward:200.0
Episode: 1968, total_reward:200.0
Episode: 1969, total_reward:200.0
Episode: 1970, total_reward:200.0
Episode: 1971, total_reward:200.0
Episode: 1972, total_reward:200.0
Episode: 1973, total_reward:200.0
Episode: 1974, total_reward:200.0
Episode: 1975, total_reward:200.0
Episode: 1976, total_reward:200.0
Episode: 1977, total_reward:200.0
Episode: 1978, total_reward:200.0
Episode: 1979, total_reward:200.0
Episode: 1980, total_reward:200.0
Episode: 1981, total_reward:200.0
Episode: 1982, total_reward:200.0
Episode: 1983, total_reward:200.0
Episode: 1984,